# 앙상블과 주가수익률 예측
앙상블 전략과 Sentiment Analysis 활용을 투자 전략에 활용하기

- 학습내용
    - 모멘텀 요인 / 한국은행 API 소개
    - 주가 자료 다운로드 / 수익률, 수준, 모멘텀 데이터 준비
    - 기초통계 (상관곤계 확인 > feature selection)
    - 선형회귀(feature selection) / 선형회귀 모형
    - 앙상블 모형과 성과 비교
- 학습목표
    - 분석에 필요한 자료를 다운로드할 수 있다
    - 앙상블 모형과 선형회귀모형을 비교하고 분석할 수 있다
    - 실제 주가수익률 예측에 적용할 수 있다

## 모멘텀 요인 (Momentum Factor)
- 물리학
    - 어떤 물체가 주어진 축을 중심으로 일어나는 회전운동을 변화시키기 어려운 정도를 나타내는 물리량
- Finance
    - 고수익을 기록한 주식들은 지속적으로 고수익을 유지하는 경향을 보인다는 개념
    
### 모멘텀 요인의 발전
- Fama-Farnach 3요인 모형 : 주식시장의 이상현상(anomalies) 설명
- Jegadeesh and Titman : 모멘텀 관련 논문 발표
    - 모멘텀 요인이 Stock Return을 설명하는 Pricing Factor로 자리매김
- Carhart : 4요인 모형에 포함
    - FF3 Factor + momentum factor
    
### 모멘텀 요인의 구분
- 절대 모멘텀
    - Absolute Momentum
    - 시계열모멘텀 (Time-series momentum)
    - 추세추종 (Trend Following)
- 상대 모멘텀
    - 기준재(numeraire) 주식을 선정하여, 상대가격의 수익률로 모멘텀을 추정
    
> 실제 투자전략 운용에서는 두 가지 개념의 구분은 크게 중요하지 않음

### 모멘텀 요인의 투자전략 적용
- 특정 기간을 비교해 기간 중 수익률에 따라 서열화
- 상위(고수익) 주식 매수
- 하위(저수익) 주식 매도(공매도)
    - Long, short 전략을 동시에 취해 위험 최소화, 수익 극대화 전략
    
## 한국은행 Open API
- 환율, 이자율 및 거시경제 관련 데이터 다운로드 ([Link](http://ecos.bok.or.kr/jsp/openapi/OpenApiController.jsp?t=main))
    - 회원가입 절차 진행 후 OPEN API 인증키 신청, 생성
    - [API 사용 참고 - 블로그 포스팅](https://bjecondata.blogspot.com/2019/08/blog-post_7.html)

In [10]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [16]:
api_key = 'T0OXEM1G4MBFJRZ1OM6M'
data_type = 'xml'
period = 'DD'
start_date = '20100101'
finish_date = '20201130'
code = [('028Y001', 'BEEA21'), ('028Y001', 'BEEA220')] # 첫번째 CD유통수익률, 두번째 KORIBOR 금리(12월물)

In [8]:
cd_return_rate_url = f'http://ecos.bok.or.kr/api/StatisticSearch/{api_key}/{data_type}/kr/1/100000/{code[0][0]}/{period}/{start_date}/{finish_date}/{code[0][1]}//'
koribor_return_rate_url = f'http://ecos.bok.or.kr/api/StatisticSearch/{api_key}/{data_type}/kr/1/100000/{code[1][0]}/{period}/{start_date}/{finish_date}/{code[1][1]}//'

In [13]:
def EcosGetData(api_key, dtype, period, start_date, finish_date, tcode, icode):
    
    url = f'http://ecos.bok.or.kr/api/StatisticSearch/{api_key}/{data_type}/kr/1/100000/{tcode}/{period}/{start_date}/{finish_date}/{icode}//'
    print(url)
    
    raw = requests.get(url)
    xml = BeautifulSoup(raw.text, 'xml')
    
    raw_data = xml.find_all("row")
    date_list = []
    value_list = []
    
    for item in raw_data:
        value = item.find("DATA_VALUE").text_encode('utf-8')
        date_str = item.find('TIME').text
        
        try:
            value = float(value)
        except:
            value = np.nan
            
        date_list.append(datetime.datetime.strptime(date_str, '%Y%m'))
        value_list.append(value)
        
    df = pd.DataFrame(index = date_list)
    df['%s' % (icode)] = value_list
    
    return df
    

In [14]:
EcosGetData(api_key, data_type, period, start_date, finish_date, code[0][0], code[0][1])

http://ecos.bok.or.kr/api/StatisticSearch/T0OXEM1G4MBFJRZ1OM6M/xml/kr/1/100000/028Y001/DD/20100101/20201231/BEEA21//


,BEEA21


In [17]:
EcosGetData(api_key, data_type, period, start_date, finish_date, '0000001', '036Y001')

http://ecos.bok.or.kr/api/StatisticSearch/T0OXEM1G4MBFJRZ1OM6M/xml/kr/1/100000/0000001/DD/20100101/20201130/036Y001//


,036Y001
